In [9]:
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_curve, roc_auc_score
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder


In [10]:

# Function to load images and labels from folder
def load_images_and_labels(folder, target_size=(224, 224)):
    images = []
    labels = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        label = subfolder
        for filename in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, filename)
            try:
                img = imread(image_path)
                if img is not None:
                    img_resized = resize(img, target_size, anti_aliasing=True)  # Resize image to target size
                    images.append(img_resized)  
                    labels.append(label)
                else:
                    print(f"Error reading image: {image_path}")
            except Exception as e:
                print(f"Error loading image: {image_path} - {str(e)}")
    return np.array(images), np.array(labels)


In [11]:
# Folder paths
train_folder = 'C:/Users/KIIT/Downloads/Cashew/Cashew/train'
test_folder ='C:/Users/KIIT/Downloads/Cashew/Cashew/test'
val_folder ='C:/Users/KIIT/Downloads/Cashew/Cashew/val'

In [12]:

# Load images and labels for training, testing, and validation
X_train, y_train = load_images_and_labels(train_folder)
X_test, y_test = load_images_and_labels(test_folder)
X_val, y_val = load_images_and_labels(val_folder)


Error loading image: C:/Users/KIIT/Downloads/Cashew/Cashew/train\red rust\red rust475_.jpg - Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image: C:/Users/KIIT/Downloads/Cashew/Cashew/train\red rust\red rust479_.jpg - Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image: C:/Users/KIIT/Downloads/Cashew/Cashew/train\red rust\red rust47_.jpg - Unable to allocate 3.66 MiB for an array with shape (400, 400, 3) and data type float64
Error loading image: C:/Users/KIIT/Downloads/Cashew/Cashew/train\red rust\red rust480_.jpg - Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64
Error loading image: C:/Users/KIIT/Downloads/Cashew/Cashew/train\red rust\red rust482_.jpg - Unable to allocate 3.66 MiB for an array with shape (400, 400, 3) and data type float64
Error loading image: C:/Users/KIIT/Downloads/Cashew/Cashew/train\red rust\red rust483_.jpg - Una

MemoryError: Unable to allocate 6.72 GiB for an array with shape (5996, 224, 224, 3) and data type float64

In [ ]:
# Convert string labels to integer labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

In [ ]:
# Load pre-trained ResNet152V2 model
base_model = ResNet152V2(weights='imagenet', include_top=False)


In [ ]:
# Add custom classifier head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(os.listdir(train_folder)), activation='softmax')(x)


In [ ]:
# Combine base model and custom head
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, recall_score
import numpy as np

# Predict probability estimates
train_pred_proba = model.predict(X_train)
val_pred_proba = model.predict(X_val)
test_pred_proba = model.predict(X_test)

# Get predicted classes
train_pred = np.argmax(train_pred_proba, axis=1)
val_pred = np.argmax(val_pred_proba, axis=1)
test_pred = np.argmax(test_pred_proba, axis=1)

# Calculate evaluation metrics
train_accuracy = accuracy_score(y_train, train_pred)
train_precision = precision_score(y_train, train_pred, average='weighted')
train_f1 = f1_score(y_train, train_pred, average='weighted')
train_recall = recall_score(y_train, train_pred, average='weighted')
train_roc_auc = roc_auc_score(y_train, train_pred_proba, average='macro', multi_class='ovr')

val_accuracy = accuracy_score(y_val, val_pred)
val_precision = precision_score(y_val, val_pred, average='weighted')
val_f1 = f1_score(y_val, val_pred, average='weighted')
val_recall = recall_score(y_val, val_pred, average='weighted')
val_roc_auc = roc_auc_score(y_val, val_pred_proba, average='macro', multi_class='ovr')

test_accuracy = accuracy_score(y_test, test_pred)
test_precision = precision_score(y_test, test_pred, average='weighted')
test_f1 = f1_score(y_test, test_pred, average='weighted')
test_recall = recall_score(y_test, test_pred, average='weighted')
test_roc_auc = roc_auc_score(y_test, test_pred_proba, average='macro', multi_class='ovr')


In [ ]:

# Print evaluation metrics
print("Train data Accuracy:", train_accuracy)
print("Train data Precision:", train_precision)
print("Train data Recall:", train_recall)
print("Train data F1 Score:", train_f1)
print("Train data ROC AUC Score:", train_roc_auc)

print("\nValidation data Accuracy:", val_accuracy)
print("Validation data Precision:", val_precision)
print("Validation data Recall:", val_recall)
print("Validation data F1 Score:", val_f1)
print("Validation data ROC AUC Score:", val_roc_auc)

print("\nTest data Accuracy:", test_accuracy)
print("Test data Precision:", test_precision)
print("Test data Recall:", test_recall)
print("Test data F1 Score:", test_f1)
print("Test data ROC AUC Score:", test_roc_auc)



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def calculate_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)
    return accuracy, precision, recall, f1, roc_auc

def print_metrics_table(train_metrics, val_metrics, test_metrics):
    print("Metrics\t\t\tTrain\t\tValidation\tTest")
    print("Accuracy\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[0], val_metrics[0], test_metrics[0]))
    print("Precision\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[1], val_metrics[1], test_metrics[1]))
    print("Recall\t\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[2], val_metrics[2], test_metrics[2]))
    print("F1 Score\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[3], val_metrics[3], test_metrics[3]))
    print("ROC AUC Score\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[4], val_metrics[4], test_metrics[4]))

# Assuming train_metrics, val_metrics, and test_metrics are tuples/lists containing metrics in the order: (accuracy, precision, recall, f1_score, roc_auc_score)
train_metrics = (train_accuracy, train_precision, train_recall, train_f1, train_roc_auc)
val_metrics = (val_accuracy, val_precision, val_recall, val_f1, val_roc_auc)
test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

print_metrics_table(train_metrics, val_metrics, test_metrics)
